In [1]:
import spotipy
import billboard
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import time
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
cid = "b3c5b350c6cb43c2bdb403eb3b896eba"
secret = "fa1aa37ba9c943d3b4e2cadc8fd4818f"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids
ids = getTrackIDs('Billboard', '6UeSakyzhiEt4NB3UAd6NQ')
print(len(ids))

100


In [3]:
def getTrackIDsFromAlbum(album_id):
    ids2 = []
    #playlist = sp.user_playlist(user, playlist_id)
    album = sp.album(album_id)
    for item in album['tracks']['items']:
        ids2.append(item['id'])
    return ids2

#ids2 = getTrackIDsFromAlbum('2Y9IRtehByVkegoD7TcLfi')
ids2 = getTrackIDsFromAlbum('0RQwQQSoR4E74jfT47ATpC')
print(len(ids2))
print(ids2)

20
['7LJeCI3v09UtUq3ETKKpgX', '5ypnp8Gi7I9duNOHaWks1r', '6ueA3RMronnNPEGgURsn2S', '4UyqE0A2NfKkPYLTTKTDqL', '6vAB19HadQGdU8yyvKRihs', '42DuSc89cXlRFzaP99W0hp', '1fFUu73Zyida3IzpNsJly9', '4pCI9LrUmH1AbIPlHPIe96', '5F38kOsptiPOVpBidD31Jz', '5tI3QwCrI0B0UYdNv7UEqa', '1wnqPz25idlXzEgI9J0hYe', '7L5ClYxMAZlpM08HkU28ro', '5es0F8fWeGdG3v7AAq6dMs', '0CkVEkQjA7tf0aGJ1UKopa', '2pUlpG7uiDk7evGOgbOC30', '4wwr9OBUij06t2BvH8yeBI', '3PildhaquXx0YNtEywZQhE', '6SAyAS35hL10OeL0jY89hS', '3IF6W3Garp7Ewfhdc28pn9', '5n1h3oQb2jKmIgOccDkuOQ']


In [4]:
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [5]:
def getTrackIdFromBillboard(name):
    results = sp.search(q="track:" + name)
    items = results["tracks"]["items"]
    id = "NONE"
    if (len(items) > 0):
        id = items[0]["id"]
    return id

In [14]:
def getChartDataFrame(chart):
    start1 = time.process_time()
    tracks = []
    for entry in chart.entries:
        id = getTrackIdFromBillboard(entry.title)
        if (id != "NONE"):
            track = getTrackFeatures(id)
            tracks.append(track)
    end1 = time.process_time()
    print("time to get billboard_chart: " + str(end1 - start1))
    start2 = time.process_time()
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
    end2 = time.process_time()
    print("time to insert tracks into dataframe: " + str(end2 - start2))
    return df


In [59]:
def gatherBillboardData():
    chart = billboard.ChartData('hot-100')
    i = 0
    charts = []
    newDataFrame = pd.DataFrame()
    while (chart.previousDate):
        chart = billboard.ChartData('hot-100', chart.previousDate)
        dataFrame = getChartDataFrame(chart)
        newDataFrame = newDataFrame.append(dataFrame)
        i += 1
        if (i == 10):
            break
    return newDataFrame

In [60]:
start1 = time.process_time()
start2 = time.clock()
dataFrame = gatherBillboardData()
end1 = time.process_time()
end2 = time.clock()

elapsed1 = end1 - start1
elapsed2 = end2 - start2
print(len(dataFrame))
print("total time elapsed using process_time: " + str(elapsed1))
print("total time elapsed using clock:        " + str(elapsed2))





C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


time to get billboard_chart: 2.109375
time to insert tracks into dataframe: 0.0
time to get billboard_chart: 2.5
time to insert tracks into dataframe: 0.015625
time to get billboard_chart: 2.609375
time to insert tracks into dataframe: 0.0
time to get billboard_chart: 2.375
time to insert tracks into dataframe: 0.0
time to get billboard_chart: 2.125
time to insert tracks into dataframe: 0.0
time to get billboard_chart: 2.09375
time to insert tracks into dataframe: 0.015625
time to get billboard_chart: 2.0625
time to insert tracks into dataframe: 0.0
time to get billboard_chart: 2.15625
time to insert tracks into dataframe: 0.0
time to get billboard_chart: 2.59375
time to insert tracks into dataframe: 0.0
time to get billboard_chart: 2.953125
time to insert tracks into dataframe: 0.0
994
total time elapsed using process_time: 36.078125
total time elapsed using clock:        318.84216679999963


C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """


In [61]:
start = time.process_time()
morn = dataFrame.drop_duplicates()
end = time.process_time()
print(len(morn))
print("elapsed time was: " + str(end - start))

196
elapsed time was: 0.015625


In [62]:
billboard.charts()

['hot-100',
 'billboard-200',
 'artist-100',
 'social-50',
 'streaming-songs',
 'radio-songs',
 'digital-song-sales',
 'on-demand-songs',
 'top-album-sales',
 'current-albums',
 'catalog-albums',
 'independent-albums',
 'soundtracks',
 'vinyl-albums',
 'billboard-global-200',
 'billboard-global-excl-us',
 'greatest-billboard-200-albums',
 'greatest-billboard-200-artists',
 'greatest-hot-100-singles',
 'greatest-hot-100-artists',
 'greatest-hot-100-songs-by-women',
 'greatest-hot-100-women-artists',
 'greatest-billboard-200-albums-by-women',
 'greatest-billboard-200-women-artists',
 'greatest-billboards-top-songs-80s',
 'greatest-billboards-top-songs-90s',
 'greatest-of-all-time-pop-songs',
 'greatest-of-all-time-pop-songs-artists',
 'greatest-adult-pop-songs',
 'greatest-adult-pop-artists',
 'greatest-country-songs',
 'greatest-country-albums',
 'greatest-country-artists',
 'greatest-of-all-time-latin-artists',
 'greatest-hot-latin-songs',
 'greatest-hot-latin-songs-artists',
 'greates